In [204]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [205]:
import numpy as np
import pandas as pd
import datetime
from stocks import StockTrades
import matplotlib.pyplot as plt
from positionslib import read_positions_file


In [206]:
!grep -v "as of" /Users/rameshsubrahmanyam/Desktop/all.csv > /Users/rameshsubrahmanyam/Desktop/all_filt.csv


In [207]:
def find_key(key, dct, default=None):
    # Filter keys in the dictionary that are less than or equal to key
    valid_keys = [k for k in dct if k >= key]
    
    if not valid_keys:
        # If no valid keys found, return None 
        return default, None
    mkey=max(valid_keys)
    # Return the pnl value corresponding to the latest date
    return dct[mkey], mkey


In [208]:
def compute_stock_pnl(symbol, start_dates, end_date, price, qty):
    start_dates=list(sorted(start_dates))
    cutoff_date=end_date - datetime.timedelta(days=30)
    end_date_s=datetime.datetime.strftime(datetime.date.today(), "%Y/%m/%d")
    trades = StockTrades(symbol, start_dates[0], end_date_s)
    trades.read_data("/Users/rameshsubrahmanyam/Desktop/all_filt.csv")
    # Access the filtered DataFrame
    if trades.df is not None:
        pass    
    else:
        print("No data matching the criteria.")
    dct, last_trade_date=trades.find_pnl(qty, price)
    pnls=[]
    sdt=None
    for dt in start_dates:
        dt1=pd.to_datetime(dt, format='%m/%d/%Y')
        if last_trade_date.date() < cutoff_date: 
            pnl=np.NaN
            edt=None
        else:
            pnl, sdt=find_key(dt1, dct, default=np.NaN)
            edt=last_trade_date
        pnls.append(pnl)
            # find the latest date in the keys of dct that precedes dt
            # and save the corresponding pnl in out_lst
            # if there is no such date, then store NaN
    if len(pnls) > 1: print(symbol, pnls, dct)
    return [pnls[0], sdt, edt]

symbol = "GOOGL"
# date_format = "%m/%d/%Y"

# # Use datetime.strptime() to parse the date string into a datetime object
# # start_date = datetime.datetime.strptime("01/01/2023", date_format).date()
# # end_date = datetime.datetime.strptime("04/17/2024", date_format).date()

current_price=190
start_date = "01/01/2024"
end_date = datetime.date.today()# "05/20/2024"

pos=13
compute_stock_pnl(symbol, [start_date], end_date, current_price, pos)




[nan, None, Timestamp('2024-07-12 00:00:00')]

In [209]:

start_date = "01/01/2024"
end_date = datetime.date.today() #"07/20/2024"
df=read_positions_file("/Users/rameshsubrahmanyam/Desktop/pos.csv")
df=df.loc[df["Security Type"] != "Option", ["Symbol", "Price", "Quantity"]]
symbols=["GLD", "ARKK", "VNQ", "BIDU", "XBI", "ITB", "BIDU", "BABA", "DXJ",
          "JACK", "XHB",  "XLK", "XLV",   "XLY",  "ADSK", "AMD", "XOP", "HD", "LULU",
          "FSLR", "APD", "MRNA", "SMH", "DELL", "AMZN", "MSFT", "AAPL", "TSLA",  "GOOGL",
          "CE", "UNP", "MIDD", "EL", "CME", "ENPH", "VTI", "VRT", "MU", 
          "TAN", "EWW", "ILMN", "MU", "JACK","AVAV",
          "ROKU", "XPO", "GOOGL", "HLT", "CBRL", "USO", "SPXL"
]
pnls=[]
for symbol in sorted(list(set(symbols))):
    #locate the row with Symbol == symbol; 
    try:
        x=df.loc[df.Symbol == symbol]
        if len(x) ==0:
            p, q=0,0
        else:
            p, q=x.Price.iloc[0], x.Quantity.iloc[0]
        # print(symbol, p, q)
        
        data=compute_stock_pnl(symbol, [start_date], end_date, p, q)
        pnls.append([symbol, p, q] + data)
    except:
        print("Skipping", symbol)
out_df=pd.DataFrame(pnls, columns=["symbol", "price", "qty", "pnl", "sdate", "edate"])
print("Total pnl", out_df.pnl.sum())
out_df

                                                                            



Total margin 481831
Total pnl -173.32000000000738


,symbol,price,qty,pnl,sdate,edate
0,AAPL,230.54,40,599.09,2024-06-11,2024-07-12
1,ADSK,0.00,0,NaN,NaT,NaT
2,AMD,0.00,0,NaN,NaT,NaT
3,AMZN,194.49,8,-41.67,2024-07-05,2024-07-09
4,APD,0.00,0,-21.34,2024-07-01,2024-07-03
5,ARKK,46.79,10,2.04,2024-07-12,2024-07-12
6,AVAV,0.00,0,-199.48,2024-06-17,2024-06-21
7,BABA,0.00,0,-32.14,2024-07-01,2024-07-03
8,BIDU,0.00,0,30.01,2024-06-18,2024-07-03
9,CBRL,42.14,-2,-51.30,2024-07-02,2024-07-12
